# Cars Plate Recognition

Neste notebook estamos utilizando o resultado do GroundTruth para treinar nosso modelo. Na etapa de treinamento estamos efetuando um Hyperparameter Tuning e escolhemos o modelo com a melhor métrica.

## Importando dependências

In [ ]:
import json, random, os, shutil, cv2, boto3, sagemaker
import IPython.display as disp
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.image as mpimg
from datetime import datetime
from time import gmtime, strftime
from PIL import Image, ImageDraw
from IPython.display import Markdown
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

In [ ]:
sagemaker_client = boto3.client('sagemaker')
s3_client = boto3.client('s3')
s3 = boto3.resource('s3')
role = get_execution_role()
sess = sagemaker.Session()
bucket_manifest = '' # bucket com o manifesto de rotulação
bucket_data = '' # bucket com as imagens
bucket_model = '' # bucket com os modelos
prefix_input = '' # prefix para os dados de treinamento
prefix_test = '' # prefix para os dados de teste

## Funções auxiliares

In [ ]:
# carrega imagem no notebook
def load_image(filename):
    with open(filename, 'rb') as imageFile:
      f = imageFile.read()
      return bytearray(f)

# desenhando a bounding box na imagem
def visualize_detection(img_file, dets, classes=[], thresh=0.6):
        img = mpimg.imread(img_file)
        plt.imshow(img)
        height = img.shape[0]
        width = img.shape[1]
        colors = dict()
        for det in dets:
            (klass, score, x0, y0, x1, y1) = det
            if score < thresh:
                continue
            cls_id = int(klass)
            print(score)
            if cls_id not in colors:
                colors[cls_id] = (random.random(), random.random(), random.random())
            xmin = int(x0 * width)
            ymin = int(y0 * height)
            xmax = int(x1 * width)
            ymax = int(y1 * height)
            rect = plt.Rectangle((xmin, ymin), xmax - xmin,
                                 ymax - ymin, fill=False,
                                 edgecolor=colors[cls_id],
                                 linewidth=3.5)
            plt.gca().add_patch(rect)
        plt.show()

# leitura do manifesto de rotulação do GroundTruth
def read_manifest_file(file_path):
    with open(file_path, 'r') as f:
        output = [json.loads(line.strip()) for line in f.readlines()]
        return output

# limpeza dos dados não rotulados no manifesto
def delete_manifest_unlabeled(output_manifest_lines, sourceref):
    clean_manifest = []
    for manifest_line in output_manifest_lines:
        if sourceref in manifest_line:
            clean_manifest.append(manifest_line)
    return clean_manifest
    
# split em treinamento e validação a partir do manifesto
def train_validation_split(labels, split_factor=0.9):
    np.random.shuffle(labels)

    dataset_size = len(labels)
    train_test_split_index = round(dataset_size*split_factor)

    train_data = labels[:train_test_split_index]
    validation_data = labels[train_test_split_index:]
    return train_data, validation_data

## Treinamento dos modelos

### Detecção de placas

In [ ]:
# carregando imagem com o algoritmo built-in object-detection
training_image = get_image_uri(sess.boto_region_name, 'object-detection', repo_version='latest')
print (training_image)

In [ ]:
# efetuando o download do arquivos de manifesto do GroundTruth
s3.Bucket(bucket_manifest).download_file('br-cars-plate/manifests/output/output.manifest', 'output-br-cars-plate.manifest')
output_manifest_lines = read_manifest_file('./output-br-cars-plate.manifest')
print(f"loaded {len(output_manifest_lines)} lines")

In [ ]:
# eliminando dados sem rótulo
clean_manifest = delete_manifest_unlabeled(output_manifest_lines, 'br-cars-plate')
print(f'labeled data: {len(clean_manifest)}')

In [ ]:
# separação 70-30 entre dados de treino e dados de validação
train_data, validation_data = train_validation_split(np.array(clean_manifest), split_factor=0.7)
print(f"training data size:{train_data.shape[0]}")
print(f"validation data size:{validation_data.shape[0]}")

In [ ]:
# criando arquivos de treino e validação
with open('train-br-cars-plate.manifest', 'w') as f:
    for line in train_data:
        f.write(json.dumps(line))
        f.write('\n')
    
with open('validation-br-cars-plate.manifest', 'w') as f:
    for line in validation_data:
        f.write(json.dumps(line))
        f.write('\n')
        
!wc -l train-br-cars-plate.manifest
!wc -l validation-br-cars-plate.manifest

In [ ]:
# efetuando upload do manifesto de treino e validação
s3_client.upload_file('train-br-cars-plate.manifest', bucket_manifest, 'br-cars-plate/manifests/train.manifest')
s3_client.upload_file('validation-br-cars-plate.manifest', bucket_manifest, 'br-cars-plate/manifests/validation.manifest')

In [ ]:
# definição dos datasets para treinamento
s3_train_data_path = f's3://{bucket_manifest}/br-cars-plate/manifests/train.manifest'
s3_validation_data_path = f's3://{bucket_manifest}/br-cars-plate/manifests/validation.manifest'
print(s3_train_data_path)
print(s3_validation_data_path)

In [ ]:
# criação do Estimator
s3_output_location = f's3://{bucket_model}/br-cars-plate'

od_model = sagemaker.estimator.Estimator(training_image, # imagem do algoritmo
                                         role, # role para treinamento
                                         train_instance_count=1,
                                         train_instance_type='ml.p3.8xlarge',
                                         train_volume_size = 1000, # tamanho do volume das instâncias de treinamento
                                         train_max_run = 360000, # tempo máximo em segundos para o treinamento
                                         input_mode = 'Pipe', # modo dos dados de entrada
                                         output_path=s3_output_location, # local de armazenamento do modelo
                                         sagemaker_session=sess)

In [ ]:
# definindo os hiperparâmetros do algoritmo
od_model.set_hyperparameters(base_network='resnet-50', # arquitetura de base da rede neural
                             num_classes=1, # número de classes
                             epochs=1000, # número de vezes que o modelo passa pelos dados de treino
                             image_shape=640, # tamanho das imagens de treinamento
                             label_width=600,
                             num_training_samples=872) # quantidade de dados para treinamento

In [ ]:
# definindo os dados de entrada e validação
train_data = sagemaker.session.s3_input(s3_data=s3_train_data_path, 
                                        distribution='FullyReplicated', 
                                        content_type='application/x-recordio', 
                                        s3_data_type='AugmentedManifestFile',
                                        record_wrapping="RecordIO",
                                        attribute_names=['source-ref', 'br-cars-plate'])

validation_data = sagemaker.session.s3_input(s3_data=s3_validation_data_path, 
                                        distribution='FullyReplicated', 
                                        content_type='application/x-recordio', 
                                        s3_data_type='AugmentedManifestFile', 
                                        record_wrapping="RecordIO",
                                        attribute_names=['source-ref', 'br-cars-plate'])

In [ ]:
# tuning de hiperparâmetros com treinamento em paralelo
hyperparameter_ranges = {'learning_rate': ContinuousParameter(0.00001, 0.5),
                         'momentum': ContinuousParameter(0.0, 0.999),
                         'weight_decay': ContinuousParameter(0.0, 0.999),
                         'mini_batch_size': IntegerParameter(8, 64),
                         'optimizer': CategoricalParameter(['sgd', 'adam', 'rmsprop', 'adadelta'])}

objective_metric_name = 'validation:mAP'

tuner = HyperparameterTuner(od_model, 
                            objective_metric_name, 
                            hyperparameter_ranges,
                            objective_type='Maximize', 
                            max_jobs=20, 
                            max_parallel_jobs=2,
                            early_stopping_type='Auto')

In [ ]:
# início do treinamento
tuning_job_name = f'br-cars-plate-{strftime("%d-%H-%M-%S", gmtime())}'
tuner.fit({'train': train_data, 'validation': validation_data}, job_name=tuning_job_name)
tuner.wait()

In [ ]:
# top 5 modelos
tuner_metrics = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)
tuner_metrics.dataframe().sort_values(['FinalObjectiveValue'], ascending=False).head(5)

In [ ]:
# status do treinamento e tempo
df = tuner_metrics.dataframe()
total_time = df['TrainingElapsedTimeSeconds'].sum() / 3600
print(f'The total training time with early stopping is {total_time} hours')
df['TrainingJobStatus'].value_counts()

In [ ]:
# treinamentos que pararam
df[df.TrainingJobStatus == 'Stopped']

## Deploy dos modelos

In [ ]:
tuner_metrics = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)
tuner_metrics.dataframe().sort_values(['FinalObjectiveValue'], ascending=False).head(5)

In [ ]:
df = tuner_metrics.dataframe()
total_time = df['TrainingElapsedTimeSeconds'].sum() / 3600
print(f'The total training time with early stopping is {total_time} hours')
df['TrainingJobStatus'].value_counts()

In [ ]:
df[df.TrainingJobStatus == 'Stopped']

In [ ]:
# deploy do modelo de detecção de placas com o objeto Estimator
od_identifier = tuner.deploy(initial_instance_count = 1,
                          instance_type = 'ml.c5.large')

## Efetuando a Inferência

In [ ]:
s3.Bucket(bucket_data).download_file('202006/test.png', 'test.png')
disp.Image('test.png', width=400)

In [ ]:
test_image = load_image('test.png')
results = od_identifier.predict(test_image)
detections = json.loads(results)
print(detections)

In [ ]:
visualize_detection('test.png', detections['prediction'], [], 0.70)